In [1]:
!pip install pyquery

You should consider upgrading via the '/Users/wxy/opt/anaconda3/bin/python -m pip install --upgrade pip' command.


In [2]:
from urllib.parse import urlencode
from pyquery import PyQuery as py
import requests
import pandas as pd

In [3]:
def get_information(since_id = '',uid = 0,):
    #X-Requested-With 用来标识Ajax请求，必须得有
    #Referer 用来指明请求来源  必须有
    #User-Agent  伪装浏览器，必须有
    headers = {'Referer': 'https://m.weibo.cn',
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36',
                'X-Requested-With': 'XMLHttpRequest'}

    params = {
                'uid': uid,
                'luicode': 10000011,
                'lfid': '231093_-_selffollowed',
                'type': int(uid),
                'value': 6620842908,
                'containerid': '231567' + str(uid),
                'since_id': since_id
    }

    # urlencode() 方法将参数转化为URL的GET请求参数
    url = 'https://m.weibo.cn/api/container/getIndex?' + urlencode(params)
    response = requests.get(url,headers = headers)
    #获得返回的 json 内容，做分析
    json_text = response.json()
    #获得 since_id 为增加页面做准备
    since_id = json_text.get('data').get('cardlistInfo').get('since_id')
    return json_text,since_id

In [5]:
def parse_json(json):
    items = json.get('data').get('cards')


    for item in items:
        item = item.get('mblog')
        weibo = {}
        weibo['发表时间'] = item.get('created_at')
        #weibo['手机类型'] = item.get('source')
        weibo['播放次数'] = item.get('page_info').get('play_count').split(r'万')[0]
        weibo['时长'] = item.get('page_info').get('media_info').get('duration')
        weibo['转发数量'] = item.get('reposts_count')
        weibo['点赞数'] = item.get('attitudes_count')
        weibo['评论数'] = item.get('comments_count')
        weibo['标题'] = item.get('page_info').get('title')
        
        yield weibo

In [7]:

#uid 你所要爬取的微博的ID，在响应的参数列表中可以得到，图中可以找到
uid = 6620842908
#p 爬取的页数
p = 5
data = []
s = get_information(since_id = "",uid = uid)
print(s[1])
for i in parse_json(s[0]):
    print(i)
    data.append(i)
#解析 JSON
#parse_json(s[0])
#输出解析后的内容
'''
for i in parse_json(s[0]):
    print(i)
'''
#多页爬取

for i in range(p):
    s = get_information(since_id = s[1],uid = uid)
    print(s[1])
    for j in parse_json(s[0]):
        print(j)
        data.append(j)

pd.DataFrame(data).to_csv('tnt2021-3-7vedio.csv',encoding="utf_8_sig")

4646237494058621
{'发表时间': 'Tue Aug 03 18:00:04 +0800 2021', '播放次数': '939', '时长': 1769.545, '转发数量': 100312, '点赞数': 731766, '评论数': 76413, '标题': '【时代少年团】街头采访之朱雀篇'}
{'发表时间': 'Mon Aug 02 18:22:32 +0800 2021', '播放次数': '945', '时长': 630.234, '转发数量': 172359, '点赞数': 751689, '评论数': 67892, '标题': '【时代少年团】TNT《“朱雀”造句游戏》'}
{'发表时间': 'Sat Jul 31 12:00:04 +0800 2021', '播放次数': '792', '时长': 14.255, '转发数量': 177734, '点赞数': 637752, '评论数': 68057, '标题': '主打曲《朱雀》预告'}
{'发表时间': 'Thu Jul 29 11:23:03 +0800 2021', '播放次数': '687', '时长': 16.577, '转发数量': 76185, '点赞数': 595816, '评论数': 64190, '标题': '主打曲《朱雀》概念预告'}
{'发表时间': 'Sun Jul 25 18:00:04 +0800 2021', '播放次数': '1373', '时长': 811.094, '转发数量': 192476, '点赞数': 1043291, '评论数': 86378, '标题': '【时代少年团】TNT兄弟们的坦白局'}
{'发表时间': 'Tue Jul 20 18:00:04 +0800 2021', '播放次数': '2363', '时长': 17.622, '转发数量': 206402, '点赞数': 802601, '评论数': 112059, '标题': '先行曲《我喜欢你》试听'}
{'发表时间': 'Mon Jul 19 18:00:04 +0800 2021', '播放次数': '972', '时长': 26.167, '转发数量': 186215, '点赞数': 668927, '评论数': 59305, '标题': '《乌托邦少年》

4606012445693372
{'发表时间': 'Sun Mar 21 18:00:04 +0800 2021', '播放次数': '460', '时长': 580.776, '转发数量': 44308, '点赞数': 305649, '评论数': 17724, '标题': '【时代少年团】广告拍摄花絮'}
{'发表时间': 'Tue Mar 16 18:00:06 +0800 2021', '播放次数': '583', '时长': 512.812, '转发数量': 42997, '点赞数': 360457, '评论数': 23731, '标题': '【时代少年团】TNT长沙美食争夺战'}
{'发表时间': 'Mon Mar 15 18:00:04 +0800 2021', '播放次数': '505', '时长': 942.892, '转发数量': 38312, '点赞数': 299075, '评论数': 13927, '标题': '【时代少年团】TNT「快乐大本营」行'}
{'发表时间': 'Sun Mar 14 18:00:04 +0800 2021', '播放次数': '505', '时长': 575.064, '转发数量': 65520, '点赞数': 322233, '评论数': 22512, '标题': '【时代少年团】TNT方言游戏'}
{'发表时间': 'Tue Mar 09 18:00:04 +0800 2021', '播放次数': '183', '时长': 854.052, '转发数量': 45968, '点赞数': 270732, '评论数': 17455, '标题': '【时代少年团】TNT「微博之夜」行'}
{'发表时间': 'Sun Mar 07 18:00:04 +0800 2021', '播放次数': '439', '时长': 532.479, '转发数量': 29039, '点赞数': 245173, '评论数': 12273, '标题': '【时代少年团】TNT《接招吧前辈》录制花絮Vol.4'}
{'发表时间': 'Sun Mar 07 00:00:50 +0800 2021', '播放次数': '551', '时长': 5556.487, '转发数量': 51477, '点赞数': 352105, '评论数': 29164